In [1]:
!nvidia-smi

Mon Apr 29 06:51:49 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.129.03             Driver Version: 535.129.03   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla P100-PCIE-16GB           Off | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0              26W / 250W |      0MiB / 16384MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

# environment

In [2]:
!pip install gdown --quiet

### code

In [3]:
%%bash
File_id="1eSGdjEsfusyZkXwsENAeZlCQE7J7dDjB"
File_name="benchmark"

gdown "https://drive.google.com/uc?id=${File_id}&confirm=t"
unzip -q "${File_name}.zip" -d "${File_name}"
# rm -f "${File_name}.zip"

Downloading...
From: https://drive.google.com/uc?id=1eSGdjEsfusyZkXwsENAeZlCQE7J7dDjB&confirm=t
To: /kaggle/working/benchmark.zip
100%|██████████| 36.8M/36.8M [00:00<00:00, 57.1MB/s]


In [4]:
%cd benchmark/

/kaggle/working/benchmark


### package

In [5]:
!pip install -r requirements.txt --quiet # --q 讓他安靜

### data

In [6]:
%%bash
File_id="1ea8mw6kYs6UbhMgSeVPR9tl-Agio2Ln9"
gdown "https://drive.google.com/uc?id=${File_id}&confirm=t"

folder="data/train/gm12878_ctcf"

mkdir -p $folder
unzip -q "gm12878_ctcf.zip" -d $folder

Downloading...
From: https://drive.google.com/uc?id=1ea8mw6kYs6UbhMgSeVPR9tl-Agio2Ln9&confirm=t
To: /kaggle/working/benchmark/gm12878_ctcf.zip
100%|██████████| 2.53G/2.53G [00:20<00:00, 121MB/s] 


In [8]:
!ls

code  data	      gm12878_ctcf.zip	scripts
conf  data_helper.py  requirements.txt	train.py


### output(補跑需要先有輸出)

In [9]:
%%bash
File_id="1whJhjltd4brTz2T92LPakYV-VVNWaCdr"
gdown "https://drive.google.com/uc?id=${File_id}&confirm=t"

Downloading...
From: https://drive.google.com/uc?id=1whJhjltd4brTz2T92LPakYV-VVNWaCdr&confirm=t
To: /kaggle/working/benchmark/output.zip
100%|██████████| 76.9M/76.9M [00:01<00:00, 62.8MB/s]


In [14]:
%%bash
folder="output"

mkdir -p $folder
unzip -q "output.zip" -d $folder

In [17]:
!ls output/gm12878_ctcf_2024.04.27_cnn_hyper-test

Lr0.001_CHs100_Hs200_Hl1_Bp1000  Lr0.001_CHs200_Hs200_Hl1_Bp1000
Lr0.001_CHs100_Hs200_Hl1_Bp2000  Lr0.001_CHs200_Hs200_Hl1_Bp2000
Lr0.001_CHs100_Hs200_Hl1_Bp3000  Lr0.001_CHs200_Hs200_Hl1_Bp3000
Lr0.001_CHs100_Hs300_Hl1_Bp1000  Lr0.001_CHs200_Hs300_Hl1_Bp1000
Lr0.001_CHs100_Hs300_Hl1_Bp2000  Lr0.001_CHs200_Hs300_Hl1_Bp2000


### run

In [ ]:
%%bash
#!/bin/bash
name="gm12878_ctcf_2024.04.27_cnn_hyper-test"

# Define configuration variables
config="conf/base-CNN.yaml"
input_folder="data/train/gm12878_ctcf"
out_folder_base="output/${name}"
input_format="onehot"

learning_rates=(0.001)
hidden_sizes=(200 300)
extractor_hidden_size=(100 200)
# hidden_layer=(1 2)
hidden_layer=(1)

bp_range=(1000 2000 3000)

# Define hyperparameters (using loops)
for lr in ${learning_rates[@]}; do
    for hs in ${hidden_sizes[@]}; do
        for hl in ${hidden_layer[@]}; do
            for bp in ${bp_range[@]}; do
                for ehs in ${extractor_hidden_size[@]}; do
                  # Construct output folder name with experiment details
                  out_folder="${out_folder_base}/Lr${lr}_CHs${ehs}_Hs${hs}_Hl${hl}_Bp${bp}"
                  echo $out_folder

                  # Train with the specified hyperparameters
                  python train.py \
                    --config "$config" \
                    --input "${input_folder}/${bp}bp.50ms.${input_format}/data.h5" \
                    --output_folder "$out_folder" \
                    --device gpu \
                    --eval_freq 1 \
                    --pin_memory_train True \
                    --data.anchor_size $bp \
                    --train.learning_rate $lr \
                    --model.classifier_hidden_size $hs \
                    --model.extractor_hidden_size $ehs \
                    --model.classifier_hidden_layer_n $hl

                  # 檢查執行結果
                  if [ $? -ne 0 ]; then
                    echo "Error: Training failed with batch size $batch_size"
                    exit 1
                  fi
                done
            done
        done
    done
done

# Sort experiment results
# python code/helper_scripts/sort_exp_result.py --folder "$out_folder_base"

In [8]:
%%bash
name="gm12878_ctcf_2024.04.27_cnn_hyper-test"
folder="output"

zip -r "${folder}/${name}.zip" "${folder}/${name}"

  adding: output/gm12878_polr2a_2024.04.24_cnn-pairs_hyper-test/ (stored 0%)

  adding: output/gm12878_polr2a_2024.04.24_cnn-pairs_hyper-test/Lr0.001_CHs100_Hs200_Hl1_Bp1000/ (stored 0%)

  adding: output/gm12878_polr2a_2024.04.24_cnn-pairs_hyper-test/Lr0.001_CHs100_Hs200_Hl1_Bp1000/configures.yaml (deflated 59%)

  adding: output/gm12878_polr2a_2024.04.24_cnn-pairs_hyper-test/Lr0.001_CHs100_Hs200_Hl1_Bp1000/log.txt (deflated 70%)


In [ ]:
# 一定要在這路徑底下才行
import os
os.chdir('/kaggle/working')

from IPython.display import FileLink
FileLink('benchmark/output/gm12878_ctcf_2024.04.27_cnn_hyper-test.zip')